In [3]:
import os

In [4]:
# get file path
cwd = os.getcwd()
file_name = "input.txt"
file_path = os.path.join(cwd, file_name)

In [5]:
# read in text
with open(file_path, 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
print("length of dataset in characters:" , len(text))

length of dataset in characters: 1115394


In [7]:
print (text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [8]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [9]:
# encoding and decodong dictionaries
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s:[stoi[c] for c in s]
decode = lambda l:''.join(itos[i] for i in l)

print (encode("hey there"))
print (decode(encode("hey there")))

[46, 43, 63, 1, 58, 46, 43, 56, 43]
hey there


In [10]:
# encode data and store it in torch.tensor
import torch
data =  torch.tensor(encode(text), dtype=torch.long)
print(data.shape , data.type)
print(data[:100])


torch.Size([1115394]) <built-in method type of Tensor object at 0x108abbac0>
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [12]:
# split into train and val
n = int(09.*len(data)) # first 90% for training , rest val
train_data = data[:n]
val_data = data[n:]

In [14]:
block_size = 8
train_data[:block_size+1]
# this block has 8 examples ., 47 after 18 ; 56 after 18,47 ; 57 after 18,47,56 and so on

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [16]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target is: {target}")

when input is tensor([18]) the target is: 47
when input is tensor([18, 47]) the target is: 56
when input is tensor([18, 47, 56]) the target is: 57
when input is tensor([18, 47, 56, 57]) the target is: 58
when input is tensor([18, 47, 56, 57, 58]) the target is: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target is: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target is: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target is: 58


In [19]:
# for GPU efficiency we have batches
torch.manual_seed(1337)
batch_size = 4 # no. of independent sequences processed in parallel
block_size = 8 # max contetx length for predictions

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)- block_size , (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1]for i in ix])
    return x,y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('--------------------------------------')
for b in range(batch_size): #batch dimension
    for t in range(block_size): # time dimenesion
        context = xb[b,:t+1]
        target = yb[b,t]
        print (f"when input is {context.tolist()} the target is: {target}")


inputs:
torch.Size([4, 8])
tensor([[59, 52, 49, 47, 52, 42,  1, 40],
        [53, 54, 43, 44, 59, 50,  1, 50],
        [27, 24, 33, 25, 26, 21, 13, 10],
        [47, 41, 43,  1, 53, 60, 43, 56]])
targets:
torch.Size([4, 8])
tensor([[52, 49, 47, 52, 42,  1, 40, 56],
        [54, 43, 44, 59, 50,  1, 50, 39],
        [24, 33, 25, 26, 21, 13, 10,  0],
        [41, 43,  1, 53, 60, 43, 56, 58]])
--------------------------------------
when input is [59] the target is: 52
when input is [59, 52] the target is: 49
when input is [59, 52, 49] the target is: 47
when input is [59, 52, 49, 47] the target is: 52
when input is [59, 52, 49, 47, 52] the target is: 42
when input is [59, 52, 49, 47, 52, 42] the target is: 1
when input is [59, 52, 49, 47, 52, 42, 1] the target is: 40
when input is [59, 52, 49, 47, 52, 42, 1, 40] the target is: 56
when input is [53] the target is: 54
when input is [53, 54] the target is: 43
when input is [53, 54, 43] the target is: 44
when input is [53, 54, 43, 44] the targe

In [20]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):
    
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self,idx,targets):
        logits = self.token_embedding_table(idx)  #(B,T,C)
        
        return logits

In [21]:
m = BigramLanguageModel(vocab_size)
out = m(xb,yb)
print(out.shape)

torch.Size([4, 8, 65])
